In [1]:
import os
os.environ["HF_DATASETS_OFFLINE"] = "1"
import datasets
import json

/Users/admin/fish/octupus-tool-call/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

ds = datasets.load_dataset("Skywork/Skywork-Reward-Preference-80K-v0.2")["train"]
ds

Using the latest cached version of the dataset since Skywork/Skywork-Reward-Preference-80K-v0.2 couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'default' at /Users/admin/.cache/huggingface/datasets/Skywork___skywork-reward-preference-80_k-v0.2/default/0.0.0/9757870d81894c90216e92536bad48f41475d5d0 (last modified on Wed Dec 11 15:55:17 2024).


Dataset({
    features: ['chosen', 'rejected', 'source'],
    num_rows: 77016
})

In [5]:
def map_func(example):
    ret = {}
    ret["conversation"] = example["chosen"][:-1]
    ret["chosen"] = example["chosen"][-1]
    ret["rejected"] = example["rejected"][-1]
    return ret

map_ds = ds.map(map_func, num_proc=10)
map_ds

Map (num_proc=10): 100%|██████████| 77016/77016 [00:01<00:00, 59711.02 examples/s]


Dataset({
    features: ['chosen', 'rejected', 'source', 'conversation'],
    num_rows: 77016
})

In [6]:
map_ds[0]

{'chosen': {'content': 'The revised code is quite close to the solution. I made some changes and added comments for clarity:\n\n```python\nfrom Bio import SeqIO\nfrom collections import defaultdict\n\nmegascaffold_dict = defaultdict(list)\nfor record in SeqIO.parse("megascaffolds.fasta", "fasta"):\n    megascaffold_dict[record.id] = record.seq\n\ngene_sequences = defaultdict(list)\nwith open("megascaffold_coordinates.gff") as f:\n    contig = \'\'\n    previous_end = 0\n    for line in f:\n        values = line.strip().split("\\t")\n        curr_contig = values[0]\n        start = int(values[3]) - 1\n        end = int(values[4])\n        gene_id = values[8].split("=")[1]\n        \n        # If we\'re on a new contig, handle the last sequence on the previous contig\n        if contig != curr_contig:\n            if previous_end != 0:\n                last_gene_seq = megascaffold_dict[contig][previous_end:]\n                gene_sequences[contig].append((f"{contig}_{len(gene_sequences[c

In [7]:
map_ds.push_to_hub("hqfx/Skywork-Reward-Preference-80K-v0.2")

Uploading the dataset shards: 100%|██████████| 1/1 [00:50<00:00, 50.77s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hqfx/Skywork-Reward-Preference-80K-v0.2/commit/8a1f0bfb41ae0e1a28fdd89493b94a1a7b5e0d66', commit_message='Upload dataset', commit_description='', oid='8a1f0bfb41ae0e1a28fdd89493b94a1a7b5e0d66', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hqfx/Skywork-Reward-Preference-80K-v0.2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hqfx/Skywork-Reward-Preference-80K-v0.2'), pr_revision=None, pr_num=None)